In [20]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch

transform = transforms.Compose([transforms.ToTensor()])
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Filter to digits 1 and 7
indices = [i for i, (x, y) in enumerate(mnist) if y in [1, 7]]
filtered = Subset(mnist, indices)

def get_loader(flatten=False, batch_size=64):
    X, y = [], []
    for img, label in filtered:
        if flatten:
            X.append(img.view(-1))
        else:
            X.append(img)
        y.append(1 if label == 7 else 0)
    X = torch.stack(X)  # Use torch.stack to combine tensors
    y = torch.tensor(y, dtype=torch.float32)
    return DataLoader(torch.utils.data.TensorDataset(X, y), batch_size=batch_size, shuffle=True)


In [21]:
import torch.nn as nn

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.linear = nn.Linear(input_size, 1)
    def forward(self, x):
        return torch.sigmoid(self.linear(x))


In [22]:
class LinearSVM(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.linear = nn.Linear(input_size, 1)
    def forward(self, x):
        return self.linear(x)

def hinge_loss(outputs, labels):
    labels = labels * 2 - 1  # Convert to -1 and 1
    return torch.mean(torch.clamp(1 - outputs.squeeze() * labels, min=0))


In [23]:
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)


In [24]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)


In [25]:
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs).squeeze()
            preds = (outputs > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    print(f"Accuracy: {acc:.4f}")
    return acc


In [32]:
# Logistic Regression
loader = get_loader(flatten=True)
model = LogisticRegressionModel(784)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
train_model(model, loader, criterion, optimizer)
evaluate_model(model, loader)

# SVM
model = LinearSVM(784)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for epoch in range(5):
    for inputs, labels in loader:
        outputs = model(inputs)
        loss = hinge_loss(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"SVM Epoch {epoch+1}, Loss: {loss.item():.4f}")
evaluate_model(model, loader)

# Neural Net
model = SimpleNN(784)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_model(model, loader, criterion, optimizer)
evaluate_model(model, loader)

# CNN
cnn_loader = get_loader(flatten=False)
model = SimpleCNN()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_model(model, cnn_loader, criterion, optimizer)
evaluate_model(model, cnn_loader)


Epoch 1, Loss: 0.1721
Epoch 2, Loss: 0.0571
Epoch 3, Loss: 0.0457
Epoch 4, Loss: 0.0398
Epoch 5, Loss: 0.0181
Accuracy: 0.9857
SVM Epoch 1, Loss: 0.0000
SVM Epoch 2, Loss: 0.0000
SVM Epoch 3, Loss: 0.0000
SVM Epoch 4, Loss: 0.0012
SVM Epoch 5, Loss: 0.0000
Accuracy: 0.9860
Epoch 1, Loss: 0.0013
Epoch 2, Loss: 0.0012
Epoch 3, Loss: 0.0040
Epoch 4, Loss: 0.0017
Epoch 5, Loss: 0.0001
Accuracy: 0.9977
Epoch 1, Loss: 0.0011
Epoch 2, Loss: 0.0001
Epoch 3, Loss: 0.0047
Epoch 4, Loss: 0.0005
Epoch 5, Loss: 0.0006
Accuracy: 0.9980


0.9980010763435073